To convert an LDraw file to graph representation:

In [1]:
from pyFiles import LDraw
g = LDraw.LDraw_to_graph('ldr_files/dataset/couch_01.ldr')
g

Using backend: pytorch


DGLGraph(num_nodes=73, num_edges=80,
         ndata_schemes={}
         edata_schemes={})

And back to LDraw:

In [2]:
g.convert_to_LDraw_and_verify('couch.ldr') # Returns if the graph is physically realizable

True

You can view LDraw files/LEGO structure renderings by downloading LDView: http://ldview.sourceforge.net/Downloads.html

The class LegoGraph.GeneratedLegoGraph inherits from dgl.DGLGraph and has useful methods for generating LegoGraphs:

In [3]:
from pyFiles import LegoGraph
g = LegoGraph.GeneratedLegoGraph()
g.add_generated_node(type=1)
g.add_generated_node(type=2)

The function takes the type of node to add, and the corresponding node label can be found in utils.LegoGraphToActionSequence:

In [4]:
from pyFiles import utils
to_sequence = utils.LegoGraphToActionSequence()
print(to_sequence.node_action_to_type[1], to_sequence.node_action_to_type[2])

Brick(2, 4) Brick(4, 2)


To add edges:

In [5]:
g.add_generated_edge(src=0, dest=1, x_shift=-2, z_shift=0)
print(g.number_of_nodes(), g.number_of_edges())

2 1


You can check the validity of a generated graph:

In [6]:
validate = utils.LegoGraphValidation()
print(validate.check_if_bricks_merged(g), validate.check_if_brick_overconstrained(g), 
      validate.check_if_missing_implied_edges(g), validate.check_if_graph_has_invalid_shift(g))

False False False False


In [21]:
g.add_generated_node(1)
g.add_generated_edge(src=0, dest=2, x_shift=-2, z_shift=0) # Same shift as before, directly overlaps brick #1
print(validate.check_if_bricks_merged(g), g.valid_graph)

True False


To obtain the missing implied edges in a graph:

In [8]:
g = LegoGraph.GeneratedLegoGraph()
g.add_generated_node(type=2)
g.add_generated_node(type=2)
g.add_generated_edge(src=0, dest=1, x_shift=-2, z_shift=0)
g.add_generated_node(type=2)
g.add_generated_edge(src=0, dest=2, x_shift=2, z_shift=0)
g.add_generated_node(type=2)
g.add_generated_edge(src=1, dest=3, x_shift=2, z_shift=0) # Creates a missing implied edge between brick 2 & 3

implied_edges = utils.ImpliedEdgesUtil()
edges = implied_edges.get_implied_edges(g)
edges

Each implied edge contains information about it:

In [9]:
edges[0].__dict__

{'src': 2,
 'dest': 3,
 'x_shift': -2,
 'z_shift': 0,
 'g': DGLGraph(num_nodes=4, num_edges=3,
          ndata_schemes={}
          edata_schemes={})}

To add missing implied edges to the graph:

In [10]:
import copy

# Method 1
g_copy_1 = copy.deepcopy(g) # Create a copy so we don't modify the original
implied_edges.add_implied_edges(g_copy_1)

# Method 2
g_copy_2 = copy.deepcopy(g)
for edge in implied_edges.get_implied_edges(g_copy_2):
    edge.insert_implied_edge()

# Method 3
g_copy_3 = copy.deepcopy(g)
edges = implied_edges.get_implied_edges(g_copy_3)
edges.add_all_edges()

print(g.number_of_edges(), g_copy_1.number_of_edges(), g_copy_2.number_of_edges(), g_copy_3.number_of_edges())

3 4 4 4


You can also get the missing implied edges from a specific node:

In [11]:
implied_edges.get_edges_implied_by_node(g, 3)

To obtain options for adding a node/edge that are guarenteed to result in a physically realizable LEGO structure:

In [13]:
# Returns list of nodes that could form a directed edge from itself to a brick of the given size
g.get_nodes_that_can_connect_underneath_of(size=(4,2))

[1, 2, 3]

In [15]:
# Same but the nodes can connect on top of a brick with the given size
g.get_nodes_that_can_connect_on_top_of(size=(4,2))

[0, 1, 2]

In [20]:
# Get valid connections for an edge from the old node to a brick with the given size
g.get_valid_connections_old_underneath(old_node=1, new_size=(4,2)) # Each sublist is of the form [x_shift, z_shift]

[[-3, -1], [-3, 0], [-3, 1], [-2, -1], [-2, 0], [-2, 1]]

In [19]:
# Get valid connections for an edge from a brick with the given size to the old node
g.get_valid_connections_old_on_top(old_node=1, new_size=(4,2))

[[3, 1], [3, 0], [3, -1], [2, 1], [2, 0], [2, -1]]

Note that these methods for obtaining valid options are extremely slow and shouldn't be used at all during training. We made it a flag during generation/inference so that we could test how checkpointed models perform when they are not allowed to create invalid LEGO structures.

In [1]:
print('adsf')

adsf


In [2]:
%run DGMG_train.py

Using backend: pytorch



loaded dataset from file
car
chair
couch
cup
flat-block
hollow-cylinder
line
pyramid
table
tall-block
tower
wall
GIN reference dataset size:  1440
loaded dataset from file
DGMG validation, train sizes: 216, 1224
train loop


KeyboardInterrupt: 